In [90]:
import ast, astor
import json

In [68]:
file_content = open('design.py', 'r').read()
ast_tree = ast.parse(file_content)

In [69]:
ast_tree

In [70]:
s = ast_tree.body[0].value.s

In [72]:
if s.startswith('@nni.architecture'):
    block = ast.parse(s[len('@nni.architecture')+1:])

In [363]:
sss='''import sys

"""@nni.architecture
{
    layer_1: {
        layer_choice: [tanh, ReLU, identity, sigmoid],
        input_candidates: [images],
        input_num: 1,
        input_aggregate: None,
        outputs: layer_1_out,
    },

    layer_2: {
        layer_choice: [tanh, ReLU, identity, sigmoid],
        input_candidates: [layer_1_out],
        input_num: 1,
        input_aggregate: None,
        outputs: layer_2_out,
    },

    layer_3: {
        layer_choice: [tanh, ReLU, identity, sigmoid],
        input_candidates: [layer_1_out, layer_2_out],
        input_num: 1,
        input_aggregate: None,
        outputs: layer_3_out,
    }
}"""
final_output = layer_3_out

'''

In [404]:
sss='''import sys
import numpy

def tanh():
    pass

def Relu():
    pass

images=load_data()

"""@nni.architecture
{
    layer_1: {
        layer_choice: [tanh, ReLU, identity, sigmoid],
        input_candidates: [images],
        input_num: 1,
        input_aggregate: None,
        outputs: layer_1_out,
    },

    layer_2: {
        layer_choice: [tanh, ReLU, identity, sigmoid],
        input_candidates: [layer_1_out],
        input_num: 1,
        input_aggregate: None,
        outputs: layer_2_out,
    },

    layer_3: {
        layer_choice: [tanh, ReLU, identity, sigmoid],
        input_candidates: [layer_1_out, layer_2_out],
        input_num: 1,
        input_aggregate: None,
        outputs: layer_3_out,
    }
}"""
final_output = layer_3_out
all = [layer_1_out, layer_2_out, layer_3_out]
if __name__ == 'main':
    func()

'''


In [450]:
class Rewrite_dict_keys(ast.NodeTransformer):
    '''from ast.Name to ast.Str'''
    def visit_Dict(self, node):
        self.generic_visit(node)
        node.keys = [ast.Str(key.id) for key in node.keys]
        keys = [key.s for key in node.keys]
        if 'outputs' in keys:
            index = keys.index('outputs')
            print('here')
            node.values[index] = ast.Str(s=node.values[index].id)
        if 'input_candidates' in keys:
            index = keys.index('input_candidates')
            node.values[index].elts = [ast.Str(s=name.id) for name in node.values[index].elts]
        
        #is_needed = lambda idx, val:isinstance(val, ast.Name) and node.keys[idx].s=='outputs'
        #node.values = [ast.Str(s=val.id) if is_needed(idx, val) else val for idx, val in enumerate(node.values)]
        
        return node

class NodeTransformer(ast.NodeTransformer):
    '''from ast.Name to ast.Str'''
    def __init__(self):
        self.rewrite_dict_keys = Rewrite_dict_keys()
        self.layer_dict_initialized = False
        self.layer_dict_name = 'nni_layer_info'
        
    def define_dict(self):
        return ast.parse(self.layer_dict_name+"=dict()").body[0]

    def make_attr_call(self, attr_name, attr_attr, args):
        '''generate an attribute call'''
        attribute = ast.Attribute(value=ast.Name(id=attr_name), attr=attr_attr)
        call_node = ast.Call(func=attribute, args=args, keywords=[])
        
        return call_node
        
    def update_dict(self, node):
        call_node = self.make_attr_call(self.layer_dict_name, 'update', [node])

        return ast.Expr(value=call_node)

    def get_layer_output(self, layer_name):
        '''generate an dict node like 'self.nni_layer_info'['layer_name']['outputs']'''
        inner_value = ast.Name(id=self.layer_dict_name, ctx=ast.Load())
        inner_slice = ast.Index(value=ast.Str(s=layer_name))
        inner_dict = ast.Subscript(value=inner_value, slice=inner_slice, ctx=ast.Load())
        
        outer_slice = ast.Index(value=ast.Str(s='outputs'))
        outer_dict = ast.Subscript(value=inner_dict, slice=outer_slice, ctx=ast.Store())

        return outer_dict

    def get_define_layer_nodes(self, dict_node):
        layer_nodes = list()
        for layer_name, info in zip(dict_node.keys, dict_node.values):
            info_keys = [key.s for key in info.keys]
            output_node = ast.Name(id=info.values[info_keys.index('outputs')].s)
            # call NNI API to get output
            args = [ast.Name(id=self.layer_dict_name), ast.Str(layer_name.s)]
            value_node = self.make_attr_call('nni','get_layer_output', args)
            assign_node = ast.Assign(targets=[output_node], value=value_node)
            layer_nodes.append(assign_node)
            
        return layer_nodes

    def visit(self, node):
        self.generic_visit(node)
        if type(node) is ast.Expr and type(node.value) is ast.Str:
            string = node.value.s
            flag = '@nni.architecture'
            if string.startswith(flag):
                return_node_list = list()
                dict_node = ast.parse(string[len(flag)+1:])
                dict_node = self.rewrite_dict_keys.visit(dict_node)
                dict_node = dict_node.body[0].value
                if not self.layer_dict_initialized:
                    self.layer_dict_initialized = True
                    return_node_list.append(self.define_dict())
                return_node_list.append(self.update_dict(dict_node))
                #layer_names = [layer_name.s for layer_name in dict_node.keys]
                return_node_list.extend(self.get_define_layer_nodes(dict_node))
                
                return (*return_node_list,)
        return node

In [451]:
nodeTransformer = NodeTransformer()
tt = nodeTransformer.visit(ast.parse(sss))

here
here
here


In [452]:
print(astor.to_source(tt))

import sys
import numpy


def tanh():
    pass


def Relu():
    pass


images = load_data()
nni_layer_info = dict()
nni_layer_info.update({'layer_1': {'layer_choice': [tanh, ReLU, identity,
    sigmoid], 'input_candidates': ['images'], 'input_num': 1,
    'input_aggregate': None, 'outputs': 'layer_1_out'}, 'layer_2': {
    'layer_choice': [tanh, ReLU, identity, sigmoid], 'input_candidates': [
    'layer_1_out'], 'input_num': 1, 'input_aggregate': None, 'outputs':
    'layer_2_out'}, 'layer_3': {'layer_choice': [tanh, ReLU, identity,
    sigmoid], 'input_candidates': ['layer_1_out', 'layer_2_out'],
    'input_num': 1, 'input_aggregate': None, 'outputs': 'layer_3_out'}})
layer_1_out = nni.get_layer_output(nni_layer_info, 'layer_1')
layer_2_out = nni.get_layer_output(nni_layer_info, 'layer_2')
layer_3_out = nni.get_layer_output(nni_layer_info, 'layer_3')
final_output = layer_3_out
all = [layer_1_out, layer_2_out, layer_3_out]
if __name__ == 'main':
    func()



In [378]:
foo(tt)

Module(
    body=[Import(names=[alias(name='sys', asname=None)]),
        Import(names=[alias(name='numpy', asname=None)]),
        FunctionDef(name='tanh',
            args=arguments(args=[], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]),
            body=[Pass],
            decorator_list=[],
            returns=None),
        FunctionDef(name='Relu',
            args=arguments(args=[], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]),
            body=[Pass],
            decorator_list=[],
            returns=None),
        Assign(targets=[Name(id='images')], value=Call(func=Name(id='load_data'), args=[], keywords=[])),
        Assign(targets=[Name(id='nni_layer_info')], value=Call(func=Name(id='dict'), args=[], keywords=[])),
        Expr(
            value=Call(func=Attribute(value=Name(id='nni_layer_info'), attr='update'),
                args=[
                    Dict(keys=[Str(s='layer_1'), Str(s='layer_2'), Str(s='layer_3')],
       

In [415]:
foo(ast.parse('{"ss":[1,2,3]}'))

Module(body=[Expr(value=Dict(keys=[Str(s='ss')], values=[List(elts=[Num(n=1), Num(n=2), Num(n=3)])]))])


In [282]:
ast.dump(ast.parse("nni_layer_info['layer_name']['outputs']"))

"Module(body=[Expr(value=Subscript(value=Subscript(value=Name(id='nni_layer_info', ctx=Load()), slice=Index(value=Str(s='layer_name')), ctx=Load()), slice=Index(value=Str(s='outputs')), ctx=Load()))])"

In [409]:
def abc(i):
    print('abc:', i)

def bcd(q):
    print('BCD: ', q)
    
dd = ['abc','bcd']

In [229]:
foo(ast.parse('a=b'))

Module(body=[Assign(targets=[Name(id='a')], value=Name(id='b'))])


In [412]:
fdd = [eval(i) for i in dd]
for i in fdd:
    i(foo)
def jj(fu):
    fu(123)
    
jj(abc)

abc: <function foo at 0x7ff10815abf8>
BCD:  <function foo at 0x7ff10815abf8>
abc: 123


In [394]:
a=1
b=2
c=3

In [408]:
eval('b')

2

In [86]:
ast.dump(ast.parse('''layer_1_input_candidate = [graph['layer_1']['input_candidates'][x] for x in nni.get_candidate('layer_1')]
if graph['layer_1']['input_aggregate'] is not None:
    layer_1_aggregated_output = graph['layer_1']['input_aggregate'](*layer_1_input_candidate,)
layer_1_output = graph['layer_1']['layer_choice'][nni.get_layer_choice('layer_1')](layer_1_aggregated_output)'''))

"Module(body=[Assign(targets=[Name(id='layer_1_input_candidate', ctx=Store())], value=ListComp(elt=Subscript(value=Subscript(value=Subscript(value=Name(id='graph', ctx=Load()), slice=Index(value=Str(s='layer_1')), ctx=Load()), slice=Index(value=Str(s='input_candidates')), ctx=Load()), slice=Index(value=Name(id='x', ctx=Load())), ctx=Load()), generators=[comprehension(target=Name(id='x', ctx=Store()), iter=Call(func=Attribute(value=Name(id='nni', ctx=Load()), attr='get_candidate', ctx=Load()), args=[Str(s='layer_1')], keywords=[]), ifs=[], is_async=0)])), If(test=Compare(left=Subscript(value=Subscript(value=Name(id='graph', ctx=Load()), slice=Index(value=Str(s='layer_1')), ctx=Load()), slice=Index(value=Str(s='input_aggregate')), ctx=Load()), ops=[IsNot()], comparators=[NameConstant(value=None)]), body=[Assign(targets=[Name(id='layer_1_aggregated_output', ctx=Store())], value=Call(func=Subscript(value=Subscript(value=Name(id='graph', ctx=Load()), slice=Index(value=Str(s='layer_1')), ctx

In [98]:
node = ast.parse('''layer_1_input_candidate = [graph['layer_1']['input_candidates'][x] for x in nni.get_candidate('layer_1')]
if graph['layer_1']['input_aggregate'] is not None:
    layer_1_aggregated_output = graph['layer_1']['input_aggregate'](*layer_1_input_candidate,)
layer_1_output = graph['layer_1']['layer_choice'][nni.get_layer_choice('layer_1')](layer_1_aggregated_output)''')
print(astor.to_source(node))

layer_1_input_candidate = [graph['layer_1']['input_candidates'][x] for x in
    nni.get_candidate('layer_1')]
if graph['layer_1']['input_aggregate'] is not None:
    layer_1_aggregated_output = graph['layer_1']['input_aggregate'](*
        layer_1_input_candidate)
layer_1_output = graph['layer_1']['layer_choice'][nni.get_layer_choice(
    'layer_1')](layer_1_aggregated_output)



In [108]:
astor.to_source("ast.parse('"djfkslfk"')")

AttributeError: No defined handler for node of type str

In [99]:
ast.dump(ast.parse("foo(*layer_1_input_candidate,)"))

"Module(body=[Expr(value=Call(func=Name(id='foo', ctx=Load()), args=[Starred(value=Name(id='layer_1_input_candidate', ctx=Load()), ctx=Load())], keywords=[]))])"

In [100]:
ast.dump(ast.parse("foo(*layer_1_input_candidate)"))

"Module(body=[Expr(value=Call(func=Name(id='foo', ctx=Load()), args=[Starred(value=Name(id='layer_1_input_candidate', ctx=Load()), ctx=Load())], keywords=[]))])"

In [106]:
def expand_layer(layer_name):
    template = '''layer_name_input_candidate = [graph['layer_name']['input_candidates'][x] for x in nni.get_candidate('layer_name')]
if graph['layer_name']['input_aggregate'] is not None:
    layer_name_aggregated_output = graph['layer_name']['input_aggregate'](*layer_name_input_candidate,)
layer_name_output = graph['layer_name']['layer_choice'][nni.get_layer_choice('layer_name')](layer_name_aggregated_output)'''
    template = template.replace('layer_name', layer_name)
    return template

In [107]:
expand_layer('fuck')

"fuck_input_candidate = [graph['fuck']['input_candidates'][x] for x in nni.get_candidate('fuck')]\nif graph['fuck']['input_aggregate'] is not None:\n    fuck_aggregated_output = graph['fuck']['input_aggregate'](*fuck_input_candidate,)\nfuck_output = graph['fuck']['layer_choice'][nni.get_layer_choice('fuck')](fuck_aggregated_output)"

In [111]:
node=ast.parse("foo(*layer_1_input_candidate,)")

In [122]:
a = node.body[0].value.args[0]
#node.body[0].value.args = [ast.Tuple(a)]

In [127]:
see=ast.Tuple(elts=[a], ctx=ast.Store())

In [128]:
ast.dump(see)

"Tuple(elts=[Starred(value=Name(id='layer_1_input_candidate', ctx=Load()), ctx=Load())], ctx=Store())"

In [129]:
node.body[0].value.args[0] = see

In [130]:
astor.to_source(node)

'foo((*layer_1_input_candidate,))\n'

In [119]:
ast.dump(ast.parse("*a, = [1,2,3,4]"))

"Module(body=[Assign(targets=[Tuple(elts=[Starred(value=Name(id='a', ctx=Store()), ctx=Store())], ctx=Store())], value=List(elts=[Num(n=1), Num(n=2), Num(n=3), Num(n=4)], ctx=Load()))])"

In [120]:
astor.to_source(ast.parse("*a, = [1,2,3,4]"))

'*a, = [1, 2, 3, 4]\n'

In [73]:
def foo(ast_tree):
    print('\n'.join(re.split('\n|\\\\n', astor.dump_tree(ast_tree))))